In [1]:
import pandas as pd

In [2]:
import json

In [2]:
from sqlalchemy import create_engine, text
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials


engine = create_engine('sqlite:///test_db.db')
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'

id = 'db3928a626e94bdd81767fa2b98adbee'
secret = '22be102e85da41dcba839a8a792b14e2'

client_credentials_manager = SpotifyClientCredentials(client_id=id , client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [42]:
# Erster vereinfachter ETL Prozess
f = open('mpd.slice.0-999.json')
data = json.load(f)

with engine.connect() as con:
    for tu in data['playlists']:
        # Get the Data for the Playlist Table
        playlist_name = tu['name']
        # collaborative = int(tu['collaborative'])
        nu_tracks = tu['num_tracks']
        # nu_albumns = tu['num_albums']
        nu_followers = tu['num_followers']
        nu_artists = tu['num_artists']
        duration_playlist = tu['duration_ms']
        nu_edits = tu['num_edits']

        statement = f'INSERT INTO Playlist(Name, Tracks, Artists, Followers, Edits, Duration) values ("{playlist_name}", {nu_tracks}, {nu_artists}, {nu_followers}, {nu_edits}, {duration_playlist})'
        con.execute(text(statement))

        # Get the Data for the Song Table
        for song in tu['tracks']:
            song_name = song['track_name'].replace("'", "")
            artist_name = song['artist_name'].replace("'", "")
            album_name = song['album_name'].replace("'", "")
            duration_song = song['duration_ms']
            uri = song['track_uri'] 
            e = sp.audio_features(uri)

            energy = e[0]['energy']
            loudness = e[0]['loudness']

            statement2 = f"INSERT INTO Song(Name, Artist, Album, duration, energy, loudness) values('{song_name}', '{artist_name}', '{album_name}', {duration_song}, {energy}, {loudness})"
            con.execute(text(statement2))
        
    

In [22]:
t

[{'pos': 0,
  'artist_name': 'Pvris',
  'track_uri': 'spotify:track:3rUTBx7gmn1IAAwsrjtnO0',
  'artist_uri': 'spotify:artist:6oFs3qk4VepIVFdoD4jmsy',
  'track_name': 'Mind over Matter (Acoustic)',
  'album_uri': 'spotify:album:33oxbqzo7poSO2IbSMqrPS',
  'duration_ms': 203649,
  'album_name': 'Acoustic'},
 {'pos': 1,
  'artist_name': 'Ingrid Michaelson',
  'track_uri': 'spotify:track:6zYqyOJuyXDOvWJwKkZ8tg',
  'artist_uri': 'spotify:artist:2vm8GdHyrJh2O2MfbQFYG0',
  'track_name': 'You and I',
  'album_uri': 'spotify:album:7o2VLivg95UduHjTMTIEIf',
  'duration_ms': 148400,
  'album_name': 'Be OK'},
 {'pos': 2,
  'artist_name': 'Troye Sivan',
  'track_uri': 'spotify:track:0nRuVPZx2iA3KWR4xSzryl',
  'artist_uri': 'spotify:artist:3WGpXCj9YhhfX11TToZcXP',
  'track_name': 'TALK ME DOWN',
  'album_uri': 'spotify:album:5ouTDazE4LF9bVJPx1nlgW',
  'duration_ms': 237464,
  'album_name': 'Blue Neighbourhood'},
 {'pos': 3,
  'artist_name': 'City and Colour',
  'track_uri': 'spotify:track:2pbxqEYiXJTv

In [4]:
data = {'Tracks' : tracks, 'albums' : albums, 'name' : name}
df = pd.DataFrame.from_dict(data)
df

,Tracks,albums,name
0,52,47,Throwbacks
1,39,23,Awesome Playlist
2,64,51,korean
3,126,107,mat
4,17,16,90s
...,...,...,...
995,41,40,old
996,17,17,Daze
997,119,98,rap
998,108,75,Country


In [5]:
df.groupby(['name'])[['Tracks']].count().sort_values(['Tracks'], ascending=False).head(20)

,Tracks
name,
Country,17
Worship,6
rap,6
country,6
Rock,6
Chill,6
chill,4
oldies,4
Workout,4


In [12]:
# Test der Spotipy Libray
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'

id = 'db3928a626e94bdd81767fa2b98adbee'
secret = '22be102e85da41dcba839a8a792b14e2'

client_credentials_manager = SpotifyClientCredentials(client_id=id , client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [4]:
e = sp.audio_analysis('spotify:track:6Lh85Pe18pqvNdDoWgTVUC')
e

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1573298749,
  'analysis_time': 58.70948,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 4888632,
  'duration': 221.70667,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.24023,
  'start_of_fade_out': 217.64062,
  'loudness': -4.432,
  'tempo': 86.006,
  'tempo_confidence': 0.273,
  'time_signature': 4,
  'time_signature_confidence': 0.916,
  'key': 5,
  'key_confidence': 0.579,
  'mode': 1,
  'mode_confidence': 0.512,
  'codestring': 'eJwtmYuVKzsOA1NxCPqLyj-xqULP7jn3jWRbTZEgCLLXfHv1tdqv_cat2Xrd9-un_eZ5t7_5fqfVb79Xo-97f9Xa79xxx3h-sY31G3OtequtX9_889Z86843f_2O4tR1Wuv7_Ua_x2NPTX_1-vh1j2Vn_-beLPuYbLf5m4VJe-5x5v2tUZwzZ8177v7xtPZbdVic0X_rveFT-ubs_duDT1_jUtjGGZe79IZBp993f2fO89vrcWY743c2H9fGjjZY3u4F9u1zzjF_d0yMmqO1c7n3vZqMl8orsOQ-5-K82pcvczFug

In [18]:
e[0]['danceability']

0.336